# Problem and background

Moscow and London are extremely famous places in the world. These huge cities differ in many ways. Being the financial hubs of Russia and Great Britain, both capitals are multicultural and attracrive for tourists. We want to explore how much they are similar or diverse speaking of a tourist opinion regarding food, accommodation, beautiful places, and many more.

Today Tourism is one of the key corners of the economy and it is more likely that the people  visits those places that have lasting culture, are developed enough to offer tourists comfortable environment and some new breath-taking experience. And now the information is so accesible regarding every place around the world which makes it easier to explore. Therefore, tourists are usually eager to travel to different places on the basis of available information, and comparing the information between the two cities can help to say which city is more popular places among tourists and the reason of its popularity.

# Data descriprion and methodology

I have selected one central borough (Central Administrative Okrug from Moscow and Inner London from London) to analyze their neighborhoods. I will employ machine learning technique -“Clustering”-to segment the neighborhoods with such types of places like restaurants, cafes, hotels, and cultural sites on the basis of each neighborhood data concerning the foot traffic in these places. This technique will help me to locate the tourist’s popular areas and hubs, and then I can judge the similarity or dissimilarity between two cities on that basis. The data exploration, analysis and visualization for both boroughs are done similarly in the same way. I will use Foursquare API to explore the data of two cities in terms of their central neighborhoods in order to solve this problem. The data includes the information about the places in each neighborhood. In the first steps I manually wrote down the names of neighborhoods from https://en.wikipedia.org/wiki/Administrative_divisions_of_Moscow and https://www.londoncouncils.gov.uk/node/1938 and used Nominatim library to receive their coordinates.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
! pip install folium==0.5.0
import folium
!pip install geocoder
import geocoder 
print('All libraries are installed')

All libraries are installed


In [2]:
london_neighborhood_names = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington',
'Royal Borough of Kensington and Chelsea','Lambeth','Lewisham','Southwark','Tower Hamlets','Wandsworth','Westminster']
moscow_neighborhood_names = ['Arbat', 'Basmanny', 'Khamovniki', 'Krasnoselsky', 'Meshchansky', 'Presnensky',
                            'Tagansky', 'Tverskoy', 'Yakimanka', 'Zamoskvorechye', 'Kitay-gorod']

In [3]:
mos = pd.DataFrame(columns=['Neighborhood', 'Latitude', 'Longitude'])
basis = ',Central Administrative Okrug,Moscow,Russia'
for i in moscow_neighborhood_names:
    neighborhood = i + basis
    geoloc = Nominatim(user_agent="moscow_explorer").geocode(neighborhood)
    lat = geoloc.latitude
    lon = geoloc.longitude
    mos.loc[-1] = [i, lat, lon]
    mos.index += 1
mos

,Neighborhood,Latitude,Longitude
10,Arbat,55.751199,37.589872
9,Basmanny,55.767281,37.669773
8,Khamovniki,55.740047,37.573958
7,Krasnoselsky,55.777447,37.654160
6,Meshchansky,55.779172,37.627989
5,Presnensky,55.762215,37.560658
4,Tagansky,55.741572,37.667063
3,Tverskoy,55.769146,37.608835
2,Yakimanka,55.738374,37.616913
1,Zamoskvorechye,55.735716,37.634572


In [4]:
basis = basis[1:]
geoloc = Nominatim(user_agent="moscow_explorer").geocode(neighborhood)
lati = geoloc.latitude
long = geoloc.longitude
map_moscow = folium.Map(location=[lati, long], zoom_start=10)
for lat, lng, neighborhood in zip(mos['Latitude'], mos['Longitude'], mos['Neighborhood']):
    label = '{}, {}'.format(neighborhood, 'Central Administrative Okrug')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow)  
map_moscow

In [41]:
lon = pd.DataFrame(columns=['Neighborhood', 'Latitude', 'Longitude'])
basis = ',Greater London,United Kingdom'
for i in london_neighborhood_names:
    neighborhood = i + basis
    geoloc = Nominatim(user_agent="london_explorer").geocode(neighborhood)
    lat = geoloc.latitude
    long = geoloc.longitude
    lon.loc[-1] = [i, lat, long]
    lon.index += 1
lon

,Neighborhood,Latitude,Longitude
11,Camden,51.542305,-0.139560
10,Greenwich,51.482084,-0.004542
9,Hackney,51.543240,-0.049362
8,Hammersmith and Fulham,51.494055,-0.225066
7,Islington,51.538429,-0.099905
6,Royal Borough of Kensington and Chelsea,51.503795,-0.200789
5,Lambeth,51.501301,-0.117287
4,Lewisham,51.462432,-0.010133
3,Southwark,51.502922,-0.103458
2,Tower Hamlets,51.525629,-0.033585


In [6]:
basis = basis[1:]
geoloc = Nominatim(user_agent="london_explorer").geocode(neighborhood)
lati = geoloc.latitude
long = geoloc.longitude
map_london = folium.Map(location=[lati, long], zoom_start=10)
for lat, lng, neighborhood in zip(lon['Latitude'], lon['Longitude'], lon['Neighborhood']):
    label = '{}, {}'.format(neighborhood, 'Inner London')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
map_london

In [43]:
mos.reset_index(inplace=True)
lon.reset_index(inplace=True)

In [44]:
lon

,index,Neighborhood,Latitude,Longitude
0,11,Camden,51.542305,-0.139560
1,10,Greenwich,51.482084,-0.004542
2,9,Hackney,51.543240,-0.049362
3,8,Hammersmith and Fulham,51.494055,-0.225066
4,7,Islington,51.538429,-0.099905
5,6,Royal Borough of Kensington and Chelsea,51.503795,-0.200789
6,5,Lambeth,51.501301,-0.117287
7,4,Lewisham,51.462432,-0.010133
8,3,Southwark,51.502922,-0.103458
9,2,Tower Hamlets,51.525629,-0.033585


In [45]:
try:
    mos.drop('index', axis = 1, inplace = True)
    lon.drop('index', axis = 1, inplace = True)
    mos.drop('level_0', axis = 1, inplace = True)
    lon.drop('level_0', axis = 1, inplace = True)
except:
    pass

In [46]:
lon

,Neighborhood,Latitude,Longitude
0,Camden,51.542305,-0.139560
1,Greenwich,51.482084,-0.004542
2,Hackney,51.543240,-0.049362
3,Hammersmith and Fulham,51.494055,-0.225066
4,Islington,51.538429,-0.099905
5,Royal Borough of Kensington and Chelsea,51.503795,-0.200789
6,Lambeth,51.501301,-0.117287
7,Lewisham,51.462432,-0.010133
8,Southwark,51.502922,-0.103458
9,Tower Hamlets,51.525629,-0.033585


In [10]:
CLIENT_ID = 'IFA0HQXU3H3YRPTJIXDYZ0XXEUENIPENUQBEK2NKW0XNA2J3' 
CLIENT_SECRET = 'YYLSPLB32W0IOMGY1BY3THZLYPKLG2DDC5G4LZIQR2IHPPG2' 
VERSION = '20180604'
LIMIT = 30

In [11]:
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, v['venue']['name'], v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',  
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [12]:
mosven = getNearbyVenues(mos['Neighborhood'],mos['Latitude'],mos['Longitude'],)
lonven = getNearbyVenues(lon['Neighborhood'],lon['Latitude'],lon['Longitude'],)

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Camden,Regent's Canal,51.541131,-0.139636,Canal
1,Camden,Crêpes à la carte,51.544510,-0.141316,French Restaurant
2,Camden,Pizzeria Di Camden,51.540920,-0.137667,Pizza Place
3,Camden,Casa Tua,51.541589,-0.138167,Italian Restaurant
4,Camden,Hidden Coffee,51.541777,-0.138658,Café
5,Camden,Colonel Fawcett,51.541923,-0.136065,Pub
6,Camden,Mamasons,51.543468,-0.141774,Ice Cream Shop
7,Camden,The Twins Coffee Shop,51.542617,-0.137683,Café
8,Camden,Domo94 Italian Restaurant,51.541427,-0.138727,Italian Restaurant
9,Camden,Poppies Fish & Chips,51.540809,-0.143996,Fish & Chips Shop


In [13]:
mosven.head()

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arbat,Corner Café & Kitchen,55.751496,37.586757,Japanese Restaurant
1,Arbat,Театр им. Вахтангова,55.749569,37.591638,Theater
2,Arbat,Кофемания,55.752156,37.588548,Coffee Shop
3,Arbat,Спортивный комплекс «Арбат»,55.750928,37.587130,Pool
4,Arbat,Arbat Street (Арбат),55.749941,37.593327,Road


In [14]:
mosven_1hot = pd.get_dummies(mosven[['Venue Category']], prefix="", prefix_sep="")
mosven_1hot['Neighborhood'] = mosven['Neighborhood'] 
fixed_columns = [mosven_1hot.columns[-1]] + list(mosven_1hot.columns[:-1])
mosven_1hot = mosven_1hot[fixed_columns]
mosven_1hot.head()

,Neighborhood,Adult Boutique,Arcade,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bath House,Beer Bar,Beer Store,Belgian Restaurant,Board Shop,Bookstore,Botanical Garden,Boutique,Burger Joint,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caucasian Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Dessert Shop,Dumpling Restaurant,Eastern European Restaurant,Escape Room,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gift Shop,Gourmet Shop,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Opera House,Other Repair Shop,Park,Pastry Shop,Pelmeni House,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Recording Studio,Restaurant,Road,Russian Restaurant,Salon / Barbershop,Sculpture Garden,Seafood Restaurant,Shopping Mall,Soccer Field,Spa,Sporting Goods Shop,Squash Court,Steakhouse,Theater,Track,Train,Tunnel,Turkish Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Arbat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arbat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Arbat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Arbat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Arbat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
mosven_gr = mosven_1hot.groupby('Neighborhood').mean().reset_index()
n_top_ven = 5
for hood in mosven_gr['Neighborhood']:
    print("----"+hood+"----")
    temp = mosven_gr[mosven_gr['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(n_top_ven), '\n')

----Arbat----
                venue  freq
0         Coffee Shop  0.10
1               Hotel  0.07
2              Museum  0.07
3  Salon / Barbershop  0.07
4             Theater  0.07 

----Basmanny----
         venue  freq
0         Park  0.10
1    Irish Pub  0.05
2      Theater  0.05
3     Pet Café  0.05
4  Escape Room  0.05 

----Khamovniki----
                venue  freq
0   Convenience Store  0.09
1            Fountain  0.09
2                Pool  0.05
3        Camera Store  0.05
4  Salon / Barbershop  0.05 

----Kitay-gorod----
          venue  freq
0          Park  0.10
1           Pub  0.07
2      Beer Bar  0.07
3   Coffee Shop  0.07
4  Burger Joint  0.07 

----Krasnoselsky----
                  venue  freq
0             Hotel Bar  0.09
1  Gym / Fitness Center  0.09
2                 Hotel  0.04
3            Hobby Shop  0.04
4          Dance Studio  0.04 

----Meshchansky----
                  venue  freq
0           Coffee Shop  0.10
1                   Spa  0.07
2  Caucasian Re

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categ = row.iloc[1:]
    row_categ_sorted = row_categ.sort_values(ascending=False)
    return row_categ_sorted.index.values[0:n_top_ven]

In [18]:
ind = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for i in np.arange(n_top_ven):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, ind[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))
mosven_sorted = pd.DataFrame(columns=columns)
mosven_sorted['Neighborhood'] = mosven_gr['Neighborhood']

for i in np.arange(mosven_gr.shape[0]):
    mosven_sorted.iloc[i, 1:] = return_most_common_venues(mosven_gr.iloc[i, :], n_top_ven)

In [19]:
mosven_gr

,Neighborhood,Adult Boutique,Arcade,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bath House,Beer Bar,Beer Store,Belgian Restaurant,Board Shop,Bookstore,Botanical Garden,Boutique,Burger Joint,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caucasian Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Dessert Shop,Dumpling Restaurant,Eastern European Restaurant,Escape Room,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gift Shop,Gourmet Shop,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Opera House,Other Repair Shop,Park,Pastry Shop,Pelmeni House,Pet Café,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Recording Studio,Restaurant,Road,Russian Restaurant,Salon / Barbershop,Sculpture Garden,Seafood Restaurant,Shopping Mall,Soccer Field,Spa,Sporting Goods Shop,Squash Court,Steakhouse,Theater,Track,Train,Tunnel,Turkish Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Arbat,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.100000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.066667,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000
1,Basmanny,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000
2,Khamovniki,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,

In [20]:
mosven_gr_cl = mosven_gr.drop('Neighborhood', 1)
k = 4
kmeans = KMeans(n_clusters=k, random_state=0).fit(mosven_gr_cl)
kmeans.labels_[0:10] 

array([1, 3, 0, 2, 1, 1, 2, 1, 1, 1], dtype=int32)

In [21]:
mos_lab1 = mos
mos_lab1['Cluster Labels'] = kmeans.labels_
mos_lab = pd.merge(mos_lab1, mosven_sorted, on='Neighborhood')
mos_lab

,Neighborhood,Latitude,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arbat,55.751199,37.589872,1,Coffee Shop,Theater,Museum,Salon / Barbershop,Hotel
1,Basmanny,55.767281,37.669773,3,Park,Hostel,Recording Studio,Historic Site,History Museum
2,Khamovniki,55.740047,37.573958,0,Convenience Store,Fountain,Spa,Other Repair Shop,Pet Store
3,Krasnoselsky,55.777447,37.654160,2,Gym / Fitness Center,Hotel Bar,Concert Hall,Athletics & Sports,Flower Shop
4,Meshchansky,55.779172,37.627989,1,Coffee Shop,Bakery,Caucasian Restaurant,Spa,Health Food Store
5,Presnensky,55.762215,37.560658,1,Coffee Shop,Board Shop,Dance Studio,Gym / Fitness Center,Yoga Studio
6,Tagansky,55.741572,37.667063,2,Bar,Health Food Store,Park,Coffee Shop,Cosmetics Shop
7,Tverskoy,55.769146,37.608835,1,Theater,Coffee Shop,Yoga Studio,Belgian Restaurant,Art Gallery
8,Yakimanka,55.738374,37.616913,1,Coffee Shop,Café,Gym / Fitness Center,Art Gallery,Restaurant
9,Zamoskvorechye,55.735716,37.634572,1,Hotel,Coffee Shop,Yoga Studio,Candy Store,Park


In [22]:
geoloc = Nominatim(user_agent="moscow_explorer").geocode('Moscow, Russia')
mos_cl = folium.Map(location=[geoloc.latitude, geoloc.longitude], zoom_start=10)
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, point, cluster in zip(mos_lab['Latitude'], mos_lab['Longitude'], mos_lab['Neighborhood'], mos_lab['Cluster Labels']):
    label = folium.Popup(str(point) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,color=rainbow[cluster-1],
        fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(mos_cl)
mos_cl

In [23]:
mos_1 = mos_lab.loc[mos_lab['Cluster Labels'] == 0, mos_lab.columns[[0] + list(range(4, mos_lab.shape[1]))]]
mos_2 = mos_lab.loc[mos_lab['Cluster Labels'] == 1, mos_lab.columns[[0] + list(range(4, mos_lab.shape[1]))]]
mos_3 = mos_lab.loc[mos_lab['Cluster Labels'] == 2, mos_lab.columns[[0] + list(range(4, mos_lab.shape[1]))]]
mos_4 = mos_lab.loc[mos_lab['Cluster Labels'] == 3, mos_lab.columns[[0] + list(range(4, mos_lab.shape[1]))]]

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Krasnoselsky,Gym / Fitness Center,Hotel Bar,Concert Hall,Athletics & Sports,Flower Shop
6,Tagansky,Bar,Health Food Store,Park,Coffee Shop,Cosmetics Shop
10,Kitay-gorod,Park,Coffee Shop,Pub,Burger Joint,Beer Bar


# Cluster 1: Khamovniki (Convenience, repair, pet stores, fountains, spa)

In [57]:
mos_1

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Khamovniki,Convenience Store,Fountain,Spa,Other Repair Shop,Pet Store


# Cluster 2: 6 neighborhoods(Coffee shops, theaters, museums, dance and yoga studios, restaurants)

In [55]:
mos_2

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arbat,Coffee Shop,Theater,Museum,Salon / Barbershop,Hotel
4,Meshchansky,Coffee Shop,Bakery,Caucasian Restaurant,Spa,Health Food Store
5,Presnensky,Coffee Shop,Board Shop,Dance Studio,Gym / Fitness Center,Yoga Studio
7,Tverskoy,Theater,Coffee Shop,Yoga Studio,Belgian Restaurant,Art Gallery
8,Yakimanka,Coffee Shop,Café,Gym / Fitness Center,Art Gallery,Restaurant
9,Zamoskvorechye,Hotel,Coffee Shop,Yoga Studio,Candy Store,Park


# Cluster 3: 3 Neighborhoods(bars, pubs, parks, gyms)

In [56]:
mos_3

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Krasnoselsky,Gym / Fitness Center,Hotel Bar,Concert Hall,Athletics & Sports,Flower Shop
6,Tagansky,Bar,Health Food Store,Park,Coffee Shop,Cosmetics Shop
10,Kitay-gorod,Park,Coffee Shop,Pub,Burger Joint,Beer Bar


# Cluster 4: Basmanny (Parks, hostels, historic sites)

In [58]:
mos_4

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Basmanny,Park,Hostel,Recording Studio,Historic Site,History Museum


In [25]:
lonven.groupby('Neighborhood').count()

,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,
Camden,30,30,30,30
Greenwich,30,30,30,30
Hackney,29,29,29,29
Hammersmith and Fulham,30,30,30,30
Islington,30,30,30,30
Lambeth,30,30,30,30
Lewisham,30,30,30,30
Royal Borough of Kensington and Chelsea,29,29,29,29
Southwark,30,30,30,30


In [26]:
lon_1hot = pd.get_dummies(lonven[['Venue Category']], prefix="", prefix_sep="")
lon_1hot['Neighborhood'] = lonven['Neighborhood']
fixed_columns = [lon_1hot.columns[-1]] + list(lon_1hot.columns[:-1])
lon_1hot = lon_1hot[fixed_columns]
lon_1hot.head()

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Beer Bar,Beer Store,Boarding House,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Canal,Canal Lock,Caribbean Restaurant,Chaat Place,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Concert Hall,Creperie,Deli / Bodega,Dessert Shop,Eastern European Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Market,Mediterranean Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Opera House,Organic Grocery,Outdoor Sculpture,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Record Shop,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Shopping Plaza,Skate Park,Soccer Field,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop,Yoga Studio
0,Camden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Camden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Camden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Camden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Camden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
lon_gr = lon_1hot.groupby('Neighborhood').mean().reset_index()
n_top_ven = 5
for hood in lon_gr['Neighborhood']:
    print("----"+hood+"----")
    temp = lon_gr[lon_gr['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(n_top_ven), '\n')

----Camden----
                venue  freq
0                 Pub  0.13
1                Café  0.10
2  Italian Restaurant  0.10
3              Market  0.07
4      Ice Cream Shop  0.07 

----Greenwich----
            venue  freq
0             Pub  0.10
1          Market  0.07
2  History Museum  0.07
3          Garden  0.07
4            Café  0.07 

----Hackney----
         venue  freq
0          Pub  0.10
1  Coffee Shop  0.10
2  Supermarket  0.07
3      Brewery  0.07
4         Café  0.07 

----Hammersmith and Fulham----
                  venue  freq
0  Gym / Fitness Center  0.10
1           Coffee Shop  0.07
2          Dessert Shop  0.03
3          Cocktail Bar  0.03
4        Clothing Store  0.03 

----Islington----
          venue  freq
0  Burger Joint  0.07
1       Theater  0.07
2        Bakery  0.07
3           Pub  0.07
4  Cocktail Bar  0.03 

----Lambeth----
                   venue  freq
0                  Hotel  0.13
1                   Café  0.10
2                    Bar  0.07
3 

In [30]:
lonven_sorted = pd.DataFrame(columns=columns)
lonven_sorted['Neighborhood'] = lon_gr['Neighborhood']
for ind in np.arange(lon_gr.shape[0]):
    lonven_sorted.iloc[ind, 1:] = return_most_common_venues(lon_gr.iloc[ind, :], n_top_ven)
lonven_sorted

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Camden,Pub,Café,Italian Restaurant,Market,Ice Cream Shop
1,Greenwich,Pub,History Museum,Garden,Market,Café
2,Hackney,Coffee Shop,Pub,Supermarket,Café,Brewery
3,Hammersmith and Fulham,Gym / Fitness Center,Coffee Shop,Clothing Store,Pub,Burrito Place
4,Islington,Pub,Burger Joint,Bakery,Theater,French Restaurant
5,Lambeth,Hotel,Café,Bar,Sandwich Place,Farmers Market
6,Lewisham,Coffee Shop,Clothing Store,Restaurant,Café,Fast Food Restaurant
7,Royal Borough of Kensington and Chelsea,Garden,Outdoor Sculpture,Park,Pub,Café
8,Southwark,Hotel,Gym / Fitness Center,Theater,Gastropub,Pub
9,Tower Hamlets,Coffee Shop,Pub,Bus Stop,Pizza Place,Burger Joint


In [32]:
lon_gr_cl = lon_gr.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(lon_gr_cl)
kmeans.labels_[0:10] 

array([0, 0, 2, 1, 0, 1, 2, 3, 1, 2], dtype=int32)

In [48]:
lon_lab1 = lon
lon_lab1['Cluster Labels'] = kmeans.labels_
lon_lab = pd.merge(lon_lab1, lonven_sorted, on='Neighborhood')
lon_lab.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Camden,51.542305,-0.139560,0,Pub,Café,Italian Restaurant,Market,Ice Cream Shop
1,Greenwich,51.482084,-0.004542,0,Pub,History Museum,Garden,Market,Café
2,Hackney,51.543240,-0.049362,2,Coffee Shop,Pub,Supermarket,Café,Brewery
3,Hammersmith and Fulham,51.494055,-0.225066,1,Gym / Fitness Center,Coffee Shop,Clothing Store,Pub,Burrito Place
4,Islington,51.538429,-0.099905,0,Pub,Burger Joint,Bakery,Theater,French Restaurant


In [50]:
geoloc = Nominatim(user_agent="moscow_explorer").geocode('Greater London,United Kingdom')
lon_cl = folium.Map(location=[geoloc.latitude, geoloc.longitude], zoom_start=11)
markers_colors = []
for lat, lon, point, cluster in zip(lon_lab['Latitude'], lon_lab['Longitude'], lon_lab['Neighborhood'], lon_lab['Cluster Labels']):
    label = folium.Popup(str(point) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,color=rainbow[cluster-1],
        fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(lon_cl)
lon_cl

In [51]:
lon_1 = lon_lab.loc[lon_lab['Cluster Labels'] == 0, lon_lab.columns[[0] + list(range(4, lon_lab.shape[1]))]]
lon_2 = lon_lab.loc[lon_lab['Cluster Labels'] == 1, lon_lab.columns[[0] + list(range(4, lon_lab.shape[1]))]]
lon_3 = lon_lab.loc[lon_lab['Cluster Labels'] == 2, lon_lab.columns[[0] + list(range(4, lon_lab.shape[1]))]]
lon_4 = lon_lab.loc[lon_lab['Cluster Labels'] == 3, lon_lab.columns[[0] + list(range(4, lon_lab.shape[1]))]]
lon_3

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Hackney,Coffee Shop,Pub,Supermarket,Café,Brewery
6,Lambeth,Hotel,Café,Bar,Sandwich Place,Farmers Market
9,Tower Hamlets,Coffee Shop,Pub,Bus Stop,Pizza Place,Burger Joint
10,Wandsworth,Pub,Clothing Store,Supermarket,Coffee Shop,Pizza Place


# Cluster 1 : 3 Neighborhoods (Pubs, restaurants, cafes, markets)

In [59]:
lon_1

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Camden,Pub,Café,Italian Restaurant,Market,Ice Cream Shop
1,Greenwich,Pub,History Museum,Garden,Market,Café
4,Islington,Pub,Burger Joint,Bakery,Theater,French Restaurant


# Cluster 2 : 3 Neighborhoods (Gyms, fitness centers, gardens, parks)

In [60]:
lon_2

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Hammersmith and Fulham,Gym / Fitness Center,Coffee Shop,Clothing Store,Pub,Burrito Place
5,Royal Borough of Kensington and Chelsea,Garden,Outdoor Sculpture,Park,Pub,Café
8,Southwark,Hotel,Gym / Fitness Center,Theater,Gastropub,Pub


# Cluster 3 : 4 Neighborhoods (Coffee shops, pubs, cafes, fast food places, supermarkets)

In [61]:
lon_3

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Hackney,Coffee Shop,Pub,Supermarket,Café,Brewery
6,Lambeth,Hotel,Café,Bar,Sandwich Place,Farmers Market
9,Tower Hamlets,Coffee Shop,Pub,Bus Stop,Pizza Place,Burger Joint
10,Wandsworth,Pub,Clothing Store,Supermarket,Coffee Shop,Pizza Place


# Cluster 4 : 2 Neighborhoods (Cofee shops, restaurants, historic sites, clothing stores)

In [62]:
lon_4

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Lewisham,Coffee Shop,Clothing Store,Restaurant,Café,Fast Food Restaurant
11,Westminster,Historic Site,Outdoor Sculpture,Plaza,Coffee Shop,Hotel


# Results

After clustering the data of the Moscow and London central neighborhoods, both cities have venues which can be explored and attract the Tourists. The neighborhoods are much similar in features like food places, bars, theatres, museums, parks etc. These cities differs in terms of the density of places where people can eat and drink: London has more of them and almost in each neighborhood.

# Recommendations

I would recommend both Moscow and London to open more cultural sites: more museums, theatres, and sculptures may attract nore visitors. Another thing I noticed is that there aren't many restaraunts of national cuisine, which could interest many tourists who want to be accustomed with the cuisine of the country they visit.

# Conclusion

The downtown Toronto and Manhattan neighborhoods have more like similar venues. As we know that every place is unique in its own way, so that’s argument is present in both neighborhoods. The dissimilarity exists in terms of some different venues and facilities but not on a larger extent.